# RECIPE APP PROJECT 


In [ ]:
import csv
from datetime import datetime, timedelta, date
import requests

# Spoonacular API details
api_url = "https://api.spoonacular.com/recipes/findByIngredients"
api_key = "3402019df6944e5380bd1eb7cd3a78e5"  # Replace with your actual Spoonacular API key

def read_ingredients(file_path='ingredients.csv'):
    ingredients = []
    try:
        with open(file_path, mode='r', newline='') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                ingredient = row['ingredient']
                expiration_date = datetime.strptime(row['expiration_date'], '%Y-%m-%d').date()
                ingredients.append({
                    'ingredient': ingredient,
                    'expiration_date': expiration_date
                })
    except FileNotFoundError:
        # File does not exist; return an empty list
        pass
    except Exception as e:
        print(f"Error reading ingredients: {e}")
    return ingredients

def get_expiring_ingredients(ingredients, days_before=3):
    expiring_soon = []
    today = date.today()
    warning_date = today + timedelta(days=days_before)
    for item in ingredients:
        if today <= item['expiration_date'] <= warning_date:
            expiring_soon.append(item)
    return expiring_soon

def notify_user(expiring_ingredients):
    if expiring_ingredients:
        print("The following ingredients are expiring soon:")
        for item in expiring_ingredients:
            days_left = (item['expiration_date'] - date.today()).days
            print(f"- {item['ingredient']} expires in {days_left} day(s) on {item['expiration_date']}")
    else:
        print("No ingredients are expiring in the next few days.")

def add_ingredient(file_path='ingredients.csv'):
    ingredient_name = input("Enter the ingredient name: ").strip()
    if not ingredient_name:
        print("Ingredient name cannot be empty.")
        return
    ingredient_name = ingredient_name.lower()

    # Ask user to choose input method
    print("Do you want to enter the expiration date or the number of days until expiration?")
    print("1. Enter expiration date")
    print("2. Enter number of days left")
    choice = input("Enter your choice (1 or 2): ").strip()

    if choice == '1':
        expiration_date_str = input("Enter the expiration date (YYYY-MM-DD): ").strip()
        try:
            expiration_date = datetime.strptime(expiration_date_str, '%Y-%m-%d').date()
            if expiration_date < date.today():
                print("The expiration date cannot be in the past.")
                return
        except ValueError:
            print("Invalid date format. Please use YYYY-MM-DD.")
            return
    elif choice == '2':
        days_left_str = input("Enter the number of days until the ingredient expires: ").strip()
        try:
            days_left = int(days_left_str)
            if days_left <= 0:
                print("Number of days must be positive.")
                return
            expiration_date = date.today() + timedelta(days=days_left)
        except ValueError:
            print("Invalid input. Please enter a positive integer for the number of days.")
            return
    else:
        print("Invalid choice. Please enter 1 or 2.")
        return

    try:
        # Check if the file exists
        file_exists = False
        try:
            with open(file_path, 'r') as csvfile:
                file_exists = True
        except FileNotFoundError:
            file_exists = False

        with open(file_path, mode='a', newline='') as csvfile:
            fieldnames = ['ingredient', 'expiration_date']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            if not file_exists:
                writer.writeheader()
            writer.writerow({'ingredient': ingredient_name, 'expiration_date': expiration_date})
        print(f"Ingredient '{ingredient_name}' added with expiration date {expiration_date}.")
    except Exception as e:
        print(f"Error adding ingredient: {e}")

def view_ingredients(ingredients):
    if ingredients:
        print("Current ingredients:")
        for item in ingredients:
            print(f"- {item['ingredient']}: Expires on {item['expiration_date']}")
    else:
        print("No ingredients found.")

# Function to get recipes from the Spoonacular API
def get_recipes_from_api(user_ingredients):
    params = {
        "ingredients": user_ingredients,
        "number": 5,             # Number of recipes to retrieve
        "ranking": 2,            # 1: Maximize used ingredients, 2: Minimize missing ingredients
        "ignorePantry": "true",  # Ignore common pantry items like water and salt
        "apiKey": api_key
    }

    try:
        response = requests.get(api_url, params=params)
        response.raise_for_status()
        recipes = response.json()
        print(f"The API returned {len(recipes)} recipes.")
        return recipes
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred while accessing the API: {http_err}")
    except requests.exceptions.RequestException as req_err:
        print(f"Request error occurred while accessing the API: {req_err}")
    except Exception as e:
        print(f"An unexpected error occurred while accessing the API: {e}")
    return []

# Function to display recipes
def display_recipes(recipes, user_ingredients):
    if recipes:
        print(f"\nRecipes found using the ingredients: {user_ingredients}\n")
        for idx, recipe in enumerate(recipes, start=1):
            print(f"--- Recipe {idx} ---")
            print(f"Title: {recipe['title']}")
            print(f"Image: {recipe['image']}")
            print(f"Recipe ID: {recipe['id']}")
            print("Used Ingredients:")
            for ingredient in recipe['usedIngredients']:
                print(f"- {ingredient['original']}")
            print("Missing Ingredients:")
            for ingredient in recipe['missedIngredients']:
                print(f"- {ingredient['original']}")
            print("\n")
    else:
        print("No recipes found with the provided ingredients.")

def find_recipes():
    ingredients = read_ingredients()
    if not ingredients:
        print("No ingredients found. Please add ingredients first.")
        return
    # Extract ingredient names
    ingredient_names = [item['ingredient'] for item in ingredients]
    user_ingredients = ','.join(ingredient_names)
    print("\nSearching for recipes using your ingredients...")
    recipes = get_recipes_from_api(user_ingredients)
    display_recipes(recipes, user_ingredients)

def remove_ingredient(file_path='ingredients.csv'):
    ingredients = read_ingredients()
    if not ingredients:
        print("No ingredients to remove.")
        return

    view_ingredients(ingredients)
    ingredient_to_remove = input("Enter the name of the ingredient to remove: ").strip().lower()
    updated_ingredients = [item for item in ingredients if item['ingredient'] != ingredient_to_remove]

    if len(updated_ingredients) == len(ingredients):
        print(f"No ingredient named '{ingredient_to_remove}' found.")
    else:
        try:
            with open(file_path, mode='w', newline='') as csvfile:
                fieldnames = ['ingredient', 'expiration_date']
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                writer.writeheader()
                for item in updated_ingredients:
                    writer.writerow({
                        'ingredient': item['ingredient'],
                        'expiration_date': item['expiration_date']
                    })
            print(f"Ingredient '{ingredient_to_remove}' has been removed.")
        except Exception as e:
            print(f"Error removing ingredient: {e}")

def main():
    while True:
        print("\nPlease choose an option:")
        print("1. Add a new ingredient")
        print("2. Check for ingredients expiring soon")
        print("3. View all ingredients")
        print("4. Remove an ingredient")
        print("5. Find recipes based on your ingredients")
        print("6. Exit")

        choice = input("Enter your choice (1-6): ").strip()

        if choice == '1':
            add_ingredient()
        elif choice == '2':
            ingredients = read_ingredients()
            if ingredients:
                expiring_ingredients = get_expiring_ingredients(ingredients)
                notify_user(expiring_ingredients)
            else:
                print("No ingredients found. Please add ingredients first.")
        elif choice == '3':
            ingredients = read_ingredients()
            view_ingredients(ingredients)
        elif choice == '4':
            remove_ingredient()
        elif choice == '5':
            find_recipes()
        elif choice == '6':
            print("Goodbye!")
            break
        else:
            print("Invalid choice. Please enter a number from 1 to 6.")

# Directly call the main() function without using __name__
main()


/Users/cha/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(



Please choose an option:
1. Add a new ingredient
2. Check for ingredients expiring soon
3. View all ingredients
4. Remove an ingredient
5. Find recipes based on your ingredients
6. Exit
